# tools

> Tools for images browsing


In [ ]:
# | default_exp tools.browser

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | export
# | hide

import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from enum import Enum
from pathlib import Path
from ds_contrib.core.paths import list_paths
from typing import ClassVar, Iterable
from pymemcache.client import base
from collections import Counter

In [ ]:
# | export


class BrowseStatus(Enum):
    """Status of an item in the browser

    Parameters
    ----------
    Enum :
        NOT_VIEWED : 0
        ADDED : 1
        REMOVED : 2
        VIEWED : 3
    """

    NOT_VIEWED = 0
    ADDED = 1
    REMOVED = 2
    VIEWED = 3

    def __str__(self):
        return self.name

    @property
    def color(self):
        """Returns the color of the item in the browser

        Returns
        -------
        str
            str representation of color of the item in the browser (conforms css)
        """
        if self == BrowseStatus.NOT_VIEWED:
            return "black"
        elif self == BrowseStatus.ADDED:
            return "green"
        elif self == BrowseStatus.REMOVED:
            return "red"
        elif self == BrowseStatus.VIEWED:
            return "lightgrey"

In [ ]:
# | export


class StatusBar:
    """
    Status bar object containing widget and storing states of statuses
    """

    def __init__(self, statuses: list[BrowseStatus]) -> None:
        """Returns new status bar object

        Parameters
        ----------
        statuses : list[BrowseStatus]
            List of statuses for each image in the browser
        """
        self._statuses = statuses.copy()
        self.position = 0
        self.statuses_per_page = 150
        self._stats: dict[BrowseStatus, int] = Counter(self._statuses)

        self._init_widgets()
        self._set_style()

        self._update_status_bar()

    def _set_style(self):
        self._style_width = "4px"
        self._style_height = "12px"

    def _init_widgets(self):
        self._status_bar_widget = widgets.HTML(
            value="",
            layout=widgets.Layout(width="90%"),
        )

    def _generate_single_status_item(self, status: BrowseStatus, selected=False) -> str:
        match status:
            case BrowseStatus.ADDED:
                color = "green"
            case BrowseStatus.REMOVED:
                color = "red"
            case BrowseStatus.VIEWED:
                color = "gray"
            case BrowseStatus.NOT_VIEWED:
                color = "lightgray"
        border = "2px solid blue" if selected else "none"
        block = f'<span style="display: inline-block; width: {self._style_width}; height: {self._style_height}; background-color: {color}; border: {border}; margin: 1px;"></span>'
        return block

    def _generate_left_arrow(self) -> str:
        """Generate a left-pointing arrow."""
        return f"""
        <div style="
            width: 0;
            height: 0;
            border-top: 5px solid transparent;
            border-bottom: 5px solid transparent;
            border-right: 10px solid blue;
            display: inline-block;
            vertical-align: center;
            margin: 10px;
        "></div>
        """

    def _generate_right_arrow(self) -> str:
        """Generate a right-pointing arrow."""
        return f"""
        <div style="
            width: 0;
            height: 0;
            border-top: 5px solid transparent;
            border-bottom: 5px solid transparent;
            border-left: 10px solid blue;
            display: inline-block;
            vertical-align: center;
            margin: 10px;
        "></div>
        """

    def _update_status_bar(self):
        visual_blocks = []

        status_bar_half_size = self.statuses_per_page // 2

        if self.position < status_bar_half_size:
            left = 0
            right = min(len(self._statuses), self.statuses_per_page)
        elif self.position > len(self._statuses) - status_bar_half_size:
            left = max(0, len(self._statuses) - self.statuses_per_page)
            right = len(self._statuses)
        else:
            left = self.position - status_bar_half_size
            right = self.position + (self.statuses_per_page - status_bar_half_size)

        left_arrow = "" if left == 0 else self._generate_left_arrow()
        right_arrow = (
            "" if right == len(self._statuses) else self._generate_right_arrow()
        )

        visual_blocks = [
            self._generate_single_status_item(
                self._statuses[i], selected=(i == self.position)
            )
            for i in range(left, right)
        ]
        # self._status_bar_widget.value = f"""
        # {''.join(visual_blocks)} <br>
        # Current Image Position: {self.position+1}/{len(self._statuses)}
        # """
        self._status_bar_widget.value = f"""
        <div style="display: flex; align-items: center;">
            {left_arrow}
            <div style="flex;">
                {''.join(visual_blocks)}
            </div>
            {right_arrow}
        </div>
        Current Image Position: {self.position+1}/{len(self._statuses)} | Page: [{left}:{right}] <br>
        Statistics: {self.get_status_str()}
        """

    def get_status_str(self) -> str:
        """Returns string with statistics of statuses

        Returns
        -------
        str
            String with statistics of statuses
        """
        overall = len(self._statuses)
        s = ""
        for k, v in self._stats.items():
            s += f"{k}: {v} ({v/overall if overall > 0 else 0:.2%}), "
        return s[:-2]

    def update_status(self, idx: int, status: BrowseStatus):
        """Update status of item at given index

        Parameters
        ----------
        idx : int
            idx of item to update status
        status : BrowseStatus
            new status
        """
        old_status = self._statuses[idx]
        self._stats[status] += 1
        self._stats[old_status] -= 1

        self._statuses[idx] = status
        self._update_status_bar()

    def move_to(self, idx: int):
        """Move highlighted selection to idx in status bar

        Parameters
        ----------
        idx : int
            Move to index
        """
        self.position = idx
        self._update_status_bar()

    @property
    def widget(self) -> widgets.HTML:
        """Returns ipywidget of status bar

        Returns
        -------
        widgets.HTML
            ipywidget of status bar
        """
        return self._status_bar_widget

In [ ]:
sb = StatusBar([BrowseStatus.NOT_VIEWED for _ in range(10)])
display(sb.widget)

HTML(value='\n        <div style="display: flex; align-items: center;">\n            \n            <div style=…

In [ ]:
# | export


class StatusStorage:
    _instance = None
    name: ClassVar[str] = "status"

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(StatusStorage, cls).__new__(cls)
            cls._instance.__init()
        return cls._instance

    def __init(self) -> None:
        # Connect to memcached server
        self.client = base.Client(("localhost", 11211))

    def key_generator(self, path: Path):
        return f"{self.name}:{str(path.absolute().resolve())}"

    def key_parser(self, key: str):
        return Path(key.split(":", 1)[1])

    def save_status(self, path: Path, status: BrowseStatus):
        self.client.set(self.key_generator(path), value=status.value)

    def load_status(self, path: Path) -> BrowseStatus | None:
        return BrowseStatus(self.client.get(self.key_generator(path)))

    def load_all_statuses(self, paths: Iterable[Path]) -> dict[Path, BrowseStatus]:
        return {
            self.key_parser(k): BrowseStatus(int(v))
            for k, v in self.client.get_multi(
                [self.key_generator(p) for p in paths]
            ).items()
        }

    def purge(self):
        self.client.flush_all()

    def dump(self):
        return self.client.stats()

    def close(self):
        self.client.close()
        self._instance = None

In [ ]:
# | export


class ImageBrowser:
    def __init__(
        self,
        paths: list[Path],
        step_sizes: tuple = (1, 5, 10, 100, 1000),
    ) -> None:
        self._image_paths = [p.absolute().resolve() for p in paths]
        self._index = 0
        self._statuses: list[BrowseStatus] = []
        self._status_storage = StatusStorage()

        self.step_size = 1
        self._step_sizes = step_sizes
        self._init_statuses()
        self._setup_gui()

    def _init_statuses(self):
        loaded_statuses = (
            self._status_storage.load_all_statuses(self._image_paths)
            if self._status_storage
            else {}
        )
        for p in self._image_paths:
            if p in loaded_statuses:
                self._statuses.append(loaded_statuses[p])
            else:
                self._statuses.append(BrowseStatus.NOT_VIEWED)

    def finalize_statuses(self, save_path: Path):
        # create pd.dataframe from statuses
        # save to csv
        data = {
            i: (self._image_paths[i].absolute(), status)
            for i, status in enumerate(self._statuses)
        }
        df = pd.DataFrame.from_dict(data, orient="index", columns=["path", "status"])
        df.to_csv(save_path)

    def _setup_gui(self):
        self.status_bar = StatusBar(self._statuses)

        self.image_name_widget = widgets.Text(
            value="", description="", disabled=True, layout=widgets.Layout(width="90%")
        )

        self.image_container = widgets.HBox(
            layout=widgets.Layout(width="100%", border="2px solid black")
        )
        # self.image_carousel_container = widgets.HBox(layout=widgets.Layout())

        self.prev_button = widgets.Button(description="← Prev")
        self.next_button = widgets.Button(description="Next →")
        self.add_remove_button = widgets.Button(description="Add")
        self.save_button = widgets.Button(description="Save")
        self.dropdown = widgets.Dropdown(
            options=self._step_sizes,
            value=self.step_size,
            description="Step size: ",
            disabled=False,
        )

        self.attach_callbacks()

    def attach_callbacks(self):
        self.prev_button.on_click(self.cb_go_prev)
        self.next_button.on_click(self.cb_go_next)
        self.add_remove_button.on_click(self.cb_add_or_remove_image)
        self.dropdown.observe(self.cb_on_dropdown_change, names="value")
        self.save_button.on_click(self.cb_select_directory)

    def __getitem__(self, index):
        return self._image_paths[index]

    def __len__(self):
        return len(self._image_paths)

    @property
    def current_image_path(self) -> Path:
        return self[self._index]

    @property
    def current_status(self) -> BrowseStatus:
        return self._statuses[self._index]

    @current_status.setter
    def current_status(self, status: BrowseStatus):
        self._statuses[self._index] = status
        if self._status_storage:
            self._status_storage.save_status(self.current_image_path, status)

    def update_status(self, status: BrowseStatus):
        self.current_status = status
        self.status_bar.update_status(self._index, status)

    def update_add_remove_button(self):
        if self.current_status == BrowseStatus.ADDED:
            self.add_remove_button.description = "Viewed"
        else:
            self.add_remove_button.description = "Add"

    def update_image_box(self):
        self.image_name_widget.value = f"[{self._index+1}] Image: {self.current_image_path} | Status: {self.current_status.name}"

    def update_image_container(self):
        # Display the main image
        with open(self.current_image_path, "rb") as f:
            main_image = widgets.Image(
                value=f.read(),
                format="png",
                # width=960,
                # height=540,
                layout=widgets.Layout(width="100%"),
            )
            self.image_container.children = [main_image]

    ## Callbacks
    def cb_go_index(self, btn, index: int):
        self._index = index
        self.status_bar.move_to(index)
        if self.current_status == BrowseStatus.NOT_VIEWED:
            self.update_status(BrowseStatus.VIEWED)
        self.update_add_remove_button()
        self.update_image_box()
        self.update_image_container()

    # Define the button click actions
    def cb_go_prev(self, btn):
        self.cb_go_index(btn, (self._index - self.step_size) % len(self))

    def cb_go_next(self, btn):
        self.cb_go_index(btn, (self._index + self.step_size) % len(self))

    def cb_add_or_remove_image(self, change):
        if self.current_status == BrowseStatus.ADDED:
            status = BrowseStatus.VIEWED
        else:
            status = BrowseStatus.ADDED
        self.update_status(status)
        self.cb_go_next(None)

    def cb_on_dropdown_change(self, change):
        """Callback function to handle changes in the dropdown value."""
        self.step_size = change.new

    def cb_select_directory(self, btn):
        # Create a root window and immediately hide it
        root = tk.Tk()
        root.withdraw()
        current_dir = Path.cwd()
        # Open the file dialog

        file_to_save = filedialog.asksaveasfilename(
            initialdir=current_dir,
            defaultextension=".csv",
            filetypes=[("Comma separated values", "*.csv"), ("All files", "*.*")],
        )
        if not file_to_save:
            print("No file selected")
        else:
            self.finalize_statuses(Path(file_to_save))
            print(f"File {file_to_save} saved")
        clear_output(wait=True)

    def browse(self):
        try:
            self.cb_go_index(None, 0)
            display(
                widgets.VBox(
                    [
                        self.image_name_widget,
                        self.image_container,
                        self.status_bar.widget,
                        widgets.HBox(
                            [
                                self.prev_button,
                                self.next_button,
                                self.add_remove_button,
                                self.dropdown,
                                self.save_button,
                            ]
                        ),
                    ],
                    layout=widgets.Layout(width="100%"),
                ),
                clear=True,
            )
        except Exception as e:
            self._status_storage.close()
            print(e)

In [ ]:
# | export


class FolderLoader:
    def __init__(self, default_folder: Path | None = None) -> None:
        self._folder = Path(default_folder) if default_folder else Path.cwd()
        # Create the button
        self.button = widgets.Button(description="Open Folder")
        self.button.on_click(self.select_folder)
        self.folder_name_widget = widgets.Text(
            value=f"`{str(self._folder)}`",
            description="Current directory",
            disabled=True,
            layout=widgets.Layout(width="100%"),
        )

    @property
    def folder(self):
        return self._folder

    def files(self, pattern=r".*\.(jpg|jpeg|png)$", recursive=False):
        files: Iterable[Path] = list_paths(
            self.folder, pattern=pattern, recursive=recursive
        )
        files: list[Path] = list(sorted(files, key=lambda x: int(x.stem)))
        return files

    def select_folder(self, button):
        # Create a root window and immediately hide it
        root = tk.Tk()
        root.withdraw()

        # Open the folder selection dialog
        folder_selected = filedialog.askdirectory(
            title="Select a folder with images", initialdir=self._folder
        )

        # Check if a folder is selected
        if not folder_selected:
            print("No folder was chosen!")
        else:
            self._folder = Path(folder_selected)
            self.folder_name_widget.value = f"`{str(self._folder)}`"

    @property
    def widget(self):
        # Display the button in the notebook
        return widgets.HBox([self.button, self.folder_name_widget])

In [ ]:
image_folder = Path("../../dev_notebooks/data/images/")
fl = FolderLoader(image_folder)
display(fl.widget, clear=True)

In [ ]:
ib = ImageBrowser(fl.files())
ib.browse()

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()